In [3]:
import requests
import re
from bs4 import BeautifulSoup
import json
import gzip
from tqdm import tqdm
import pandas as pd

In [3]:
body = requests.get('https://www.bbc.com/').text

In [4]:
soup = BeautifulSoup(body, 'html.parser')

In [5]:
a = soup.get_text(' ').replace('\n', '')

In [6]:
b = ' '.join(a.split())

In [7]:
b = re.sub(r"[^A-Za-z ]+","", a)

In [8]:
c = ' '.join(b.split())

In [ ]:
def hook(obj):
    value = obj.get("features")
    if value:
    pbar = tqdm(value)
    for item in pbar:
        pass
        pbar.set_description("Loading")
    return obj

In [ ]:
#with gzip.open("../data/websites_40000_5cat_html.json.gz") as f:
    #data = json.load(f)

In [ ]:
pd.read_json("../data/websites_40000_5cat_html.json.gz", lines=True)